# Sandbox
For playing around during development.

In [7]:
%pylab inline
#import pysd
#print pysd.__version__
#print pysd.__file__
#import pandas as pd
#import os
import xarray as xr
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [21]:
import pysd

In [22]:
pysd.__version__

'0.7.1'

In [23]:
model = pysd.read_vensim('tests/test-models/samples/teacup/teacup.mdl')

In [24]:
model.doc()

,Real Name,Py Name,Unit,Comment
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN


In [5]:
def myfunc():
    return [func() for func in myfunc.subfuncs]
myfunc.subfuncs = []

myfunc.subfuncs.append(lambda: 5)
myfunc.subfuncs.append(lambda: 6)

In [6]:
myfunc()



[5, 6]

In [13]:
a = [xr.DataArray(range(3)), xr.DataArray(range(5))]
a

[<xarray.DataArray (dim_0: 3)>
 array([0, 1, 2])
 Coordinates:
   * dim_0    (dim_0) int64 0 1 2, <xarray.DataArray (dim_0: 5)>
 array([0, 1, 2, 3, 4])
 Coordinates:
   * dim_0    (dim_0) int64 0 1 2 3 4]

In [14]:
b = np.array(a)
b

array([ <xarray.DataArray (dim_0: 3)>
array([0, 1, 2])
Coordinates:
  * dim_0    (dim_0) int64 0 1 2,
       <xarray.DataArray (dim_0: 5)>
array([0, 1, 2, 3, 4])
Coordinates:
  * dim_0    (dim_0) int64 0 1 2 3 4], dtype=object)

In [16]:
c = b*2
c

array([ <xarray.DataArray (dim_0: 3)>
array([0, 2, 4])
Coordinates:
  * dim_0    (dim_0) int64 0 1 2,
       <xarray.DataArray (dim_0: 5)>
array([0, 2, 4, 6, 8])
Coordinates:
  * dim_0    (dim_0) int64 0 1 2 3 4], dtype=object)

In [ ]:
order = 3
init = 

In [18]:
d = np.roll(b, 1)
d[0] = xr.DataArray(range(2))
d

array([ <xarray.DataArray (dim_0: 2)>
array([0, 1])
Coordinates:
  * dim_0    (dim_0) int64 0 1,
       <xarray.DataArray (dim_0: 3)>
array([0, 1, 2])
Coordinates:
  * dim_0    (dim_0) int64 0 1 2], dtype=object)

In [19]:
getattr(a, 'hi')

AttributeError: 'list' object has no attribute 'hi'

In [20]:
a.update()

AttributeError: 'list' object has no attribute 'update'